In [59]:
## Import python library
import pandas as pd
import nltk

## Read file
file_name = '/content/Samsung Galaxy A12.csv'
## Read file using pandas
df = pd.read_csv(file_name)

In [60]:
# imports
import re
# function to remove special characters
def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, '', text)

In [61]:
import re
l = []
asd_train = []
pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'
for i in range(df.shape[0]):
    
    g = str(df.iloc[i]['body'])
    g = g.replace('[^\w\s]','')
    g = remove_special_characters(g)
    g = g.replace("/"," ")
    g = g.replace("."," ").replace(","," ").replace("\'","").replace("\"","").replace(":"," ")
    g = " ".join(g.split())
    k = ""
    for j in g:
        if(j == " "):
            k = k + j
        elif(ord(j)>=ord('A') and ord(j)<=ord('Z')):
            k = k + j
        elif(ord(j)>=ord('a') and ord(j)<=ord('z')):
            k = k + j
                
    g1 = df.iloc[i]['rating']
    if(g1>3):
        asd_train.append(1)
    else:
        asd_train.append(0)
    l.append(k)

In [62]:
df['body1'] = l

In [63]:
df['rating'] = asd_train

In [64]:
df.head()

,product,title,rating,body,body1
0,"Samsung Galaxy A12 (Black, 4GB RAM, 64GB Stora...",Dont buy dont bully urself,0,Buyd on 20th looking 128gb and good camera but...,Buyd on th looking gb and good camera but real...
1,"Samsung Galaxy A12 (Black, 4GB RAM, 64GB Stora...",Don't buy Samsung A12. This model has poor per...,0,Though its brand new phone. There is lagging i...,Though its brand new phone There is lagging in...
2,"Samsung Galaxy A12 (Black, 4GB RAM, 64GB Stora...",Nice new look,1,Nice new lookNice cameraNice & clear speaker3....,Nice new lookNice cameraNice clear speaker Ja...
3,"Samsung Galaxy A12 (Black, 4GB RAM, 64GB Stora...",never buy this product,0,This phone heats up within 10 minutes of video...,This phone heats up within minutes of video c...
4,"Samsung Galaxy A12 (Black, 4GB RAM, 64GB Stora...",Good,1,It is very smooth,It is very smooth


In [65]:
df.rating.value_counts()

0    168
1    166
Name: rating, dtype: int64

In [66]:
import nltk
nltk.download('stopwords')
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [67]:
stop = nltk.corpus.stopwords.words('english')

In [68]:
## Tokenizing sentence into token for finding synonym.
def make_tokenizer(texts):
    from keras.preprocessing.text import Tokenizer
    t = Tokenizer()
    t.fit_on_texts(texts)
    return t

tokenizer = make_tokenizer(df['body1'])    ## Message is column name

X = tokenizer.texts_to_sequences(df['body1'])

from keras_preprocessing.sequence import pad_sequences
X = pad_sequences(X, 70)

In [69]:
## Dictionary of word index
index_word = {}
for word in tokenizer.word_index.keys():
    index_word[tokenizer.word_index[word]] = word

In [70]:
## word list
words = [value for key, value in index_word.items()]

In [71]:
## Function to find synonym of words 
import spacy
nlp = spacy.load('en_core_web_sm',disable=['parser'])
def check_lemma(t,w) :
    r = [d for d in t if (nlp(d.text)[0].lemma_ != nlp(w.text)[0].lemma_)]
    return r

def get_word_synonym(word):
    filtered_words = [w for w in word.vocab if (not w.lower_ in stop) and w.is_lower == word.is_lower and w.prob >= -15] ## (not w.lower_ in stop) and
    similarity = sorted(filtered_words, key=lambda w: word.similarity(w), reverse=True)
    filtered_similarity = check_lemma(similarity[:30], word)
    return filtered_similarity[:3]

/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


In [72]:
## Synonym dictionary
synonym_dict = {}

for word in words:
    synonym_dict.update({word : tuple([w.lower_ for w in get_word_synonym(nlp.vocab[word])])})
        #print(word, " : ", [w.lower_ for w in get_word_synonym(nlp.vocab[word])])

In [73]:
## Only consider filtered synonym
import collections
value_occurrences = collections.Counter(synonym_dict.values())

filtered_synonym = {key: value for key, value in synonym_dict.items() if value_occurrences[value] == 1}

In [74]:
## Function for augmenting data by replacing words with synonym using spaCy
## This might not be best best method compared to data augmentation using language translation

import re
import random
sr = random.SystemRandom()
split_pattern = re.compile(r'\s+')
def data_augmentation(message, aug_range=1) :
    augmented_messages = []
    for j in range(0,aug_range) :
        new_message = ""
        for i in filter(None, split_pattern.split(message)) :
            new_message = new_message + " " + sr.choice(filtered_synonym.get(i,[i]))
        augmented_messages.append(new_message)
    return augmented_messages

In [75]:
## Dictionary for intent count
## Intent is column name
intent_count = df.rating.value_counts().to_dict()

In [76]:
## Get max intent count to match other minority classes through data augmentation
import operator
max_intent_count = max(intent_count.items(), key=operator.itemgetter(1))[1]

In [77]:
## Loop to interate all messages
import numpy as np
import math
import tqdm
newdf = pd.DataFrame()
for intent, count in intent_count.items() :
    count_diff = max_intent_count - count    ## Difference to fill
    multiplication_count = math.ceil((count_diff)/count)  ## Multiplying a minority classes for multiplication_count times
    if (multiplication_count) :
        old_message_df = pd.DataFrame()
        new_message_df = pd.DataFrame()
        for message in tqdm.tqdm(df[df["rating"] == intent]["body1"]) :
            ## Extracting existing minority class batch
            dummy1 = pd.DataFrame([message], columns=['body1'])
            dummy1["rating"] = intent
            old_message_df = old_message_df.append(dummy1)
            
            ## Creating new augmented batch from existing minority class
            new_messages = data_augmentation(message, multiplication_count)
            dummy2 = pd.DataFrame(new_messages, columns=['body1'])
            dummy2["rating"] = intent
            new_message_df = new_message_df.append(dummy2)
        
        ## Select random data points from augmented data
        new_message_df=new_message_df.take(np.random.permutation(len(new_message_df))[:count_diff])
        
        ## Merge existing and augmented data points
        newdf = newdf.append([old_message_df,new_message_df])
    else :
        newdf = newdf.append(df[df["rating"] == intent])

100%|██████████| 166/166 [00:00<00:00, 424.59it/s]


In [78]:
newdf.rating.value_counts()

0    168
1    168
Name: rating, dtype: int64

In [79]:
newdf.head()

,product,title,rating,body,body1
0,"Samsung Galaxy A12 (Black, 4GB RAM, 64GB Stora...",Dont buy dont bully urself,0,Buyd on 20th looking 128gb and good camera but...,Buyd on th looking gb and good camera but real...
1,"Samsung Galaxy A12 (Black, 4GB RAM, 64GB Stora...",Don't buy Samsung A12. This model has poor per...,0,Though its brand new phone. There is lagging i...,Though its brand new phone There is lagging in...
3,"Samsung Galaxy A12 (Black, 4GB RAM, 64GB Stora...",never buy this product,0,This phone heats up within 10 minutes of video...,This phone heats up within minutes of video c...
5,"Samsung Galaxy A12 (Black, 4GB RAM, 64GB Stora...",DO NOT BUY,0,Startup is slower then my 2009 nokia.Phone ove...,Startup is slower then my nokia Phone overhea...
6,"Samsung Galaxy A12 (Black, 4GB RAM, 64GB Stora...",Amazon fault,0,"Phone quality is very poor, mobile is hanging...",Phone quality is very poor mobile is hanging w...


In [80]:
newdf.tail()

,product,title,rating,body,body1
0,NaN,NaN,1,NaN,Grt
0,NaN,NaN,1,NaN,Nice phone
0,NaN,NaN,1,NaN,Its a good phone Useful and handy Good quality...
0,NaN,NaN,1,NaN,MPMPMPMP Multi Role Quad CameraMP F main came...
0,NaN,NaN,1,NaN,Amazing battery life


In [81]:
newdf.to_csv("Samsung Galaxy A12 Balanced.csv")

In [82]:
from google.colab import files
files.download('Samsung Galaxy A12 Balanced.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>